In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,cross_val_score,GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix,classification_report,confusion_matrix,precision_score,roc_curve
import seaborn as sns
from sklearn.utils import shuffle
# from pandas_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeClassifier, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier 
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics


In [76]:
df = pd.read_csv('./Data/dataset.csv')
df = shuffle(df,random_state=42)
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
373,Acne,skin_rash,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1550,Hyperthyroidism,fatigue,mood_swings,weight_loss,restlessness,sweating,diarrhoea,fast_heart_rate,excessive_hunger,muscle_weakness,irritability,abnormal_menstruation,NaN,NaN,NaN,NaN,NaN,NaN
3081,AIDS,muscle_wasting,patches_in_throat,high_fever,extra_marital_contacts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3857,Chronic cholestasis,itching,vomiting,yellowish_skin,nausea,loss_of_appetite,abdominal_pain,yellowing_of_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
for col in df.columns:
    
    df[col] = df[col].str.replace('_',' ')
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
373,Acne,skin rash,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4916,Acne,skin rash,pus filled pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1550,Hyperthyroidism,fatigue,mood swings,weight loss,restlessness,sweating,diarrhoea,fast heart rate,excessive hunger,muscle weakness,irritability,abnormal menstruation,NaN,NaN,NaN,NaN,NaN,NaN
3081,AIDS,muscle wasting,patches in throat,high fever,extra marital contacts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3857,Chronic cholestasis,itching,vomiting,yellowish skin,nausea,loss of appetite,abdominal pain,yellowing of eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df.describe()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
count,4920,4920,4920,4920,4572,3714,2934,2268,1944,1692,1512,1194,744,504,306,240,192,72
unique,41,34,48,54,50,38,32,26,21,22,21,18,11,8,4,3,3,1
top,Acne,vomiting,vomiting,fatigue,high fever,headache,nausea,abdominal pain,abdominal pain,yellowing of eyes,yellowing of eyes,irritability,malaise,stomach bleeding,chest pain,chest pain,loss of smell,muscle pain
freq,120,822,870,726,378,348,390,264,276,228,198,120,126,72,96,144,72,72


In [79]:
null_checker = df.apply(lambda x: sum(x.isnull())).to_frame(name='count')
print(null_checker)

            count
Disease         0
Symptom_1       0
Symptom_2       0
Symptom_3       0
Symptom_4     348
Symptom_5    1206
Symptom_6    1986
Symptom_7    2652
Symptom_8    2976
Symptom_9    3228
Symptom_10   3408
Symptom_11   3726
Symptom_12   4176
Symptom_13   4416
Symptom_14   4614
Symptom_15   4680
Symptom_16   4728
Symptom_17   4848


In [80]:
cols = df.columns
data = df[cols].values.flatten()

s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)

df = pd.DataFrame(s, columns=df.columns)
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Acne,skin rash,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acne,skin rash,pus filled pimples,blackheads,scurring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hyperthyroidism,fatigue,mood swings,weight loss,restlessness,sweating,diarrhoea,fast heart rate,excessive hunger,muscle weakness,irritability,abnormal menstruation,NaN,NaN,NaN,NaN,NaN,NaN
3,AIDS,muscle wasting,patches in throat,high fever,extra marital contacts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chronic cholestasis,itching,vomiting,yellowish skin,nausea,loss of appetite,abdominal pain,yellowing of eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
df = df.fillna(0)
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Acne,skin rash,blackheads,scurring,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Acne,skin rash,pus filled pimples,blackheads,scurring,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hyperthyroidism,fatigue,mood swings,weight loss,restlessness,sweating,diarrhoea,fast heart rate,excessive hunger,muscle weakness,irritability,abnormal menstruation,0,0,0,0,0,0
3,AIDS,muscle wasting,patches in throat,high fever,extra marital contacts,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Chronic cholestasis,itching,vomiting,yellowish skin,nausea,loss of appetite,abdominal pain,yellowing of eyes,0,0,0,0,0,0,0,0,0,0


In [82]:
df1 = pd.read_csv('./Data/Symptom-severity.csv')
x=df1['Symptom']
x

0                   itching
1                 skin_rash
2      nodal_skin_eruptions
3       continuous_sneezing
4                 shivering
               ...         
128      inflammatory_nails
129                 blister
130    red_sore_around_nose
131       yellow_crust_ooze
132               prognosis
Name: Symptom, Length: 133, dtype: object

In [83]:
dfx=pd.DataFrame()
dfx["Disease"]=df["Disease"]
y=0
dfx[x]=0
for index, row in df.iterrows():
    for symptom in df.columns[1:]:
        if row[symptom] != 0:
            dfx.loc[index, row[symptom]] = 1
dfx = dfx.fillna(0)
dfx[dfx.columns[1:]]=dfx[dfx.columns[1:]].astype('int')

C:\Users\Hp\AppData\Local\Temp\ipykernel_22504\3337311018.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfx[x]=0
C:\Users\Hp\AppData\Local\Temp\ipykernel_22504\3337311018.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfx[x]=0
C:\Users\Hp\AppData\Local\Temp\ipykernel_22504\3337311018.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fra

In [84]:
dfx.columns = dfx.columns.str.strip()

In [85]:
dfx

,Disease,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,...,yellow crust ooze,ulcers on tongue,spotting urination,pain behind the eyes,red spots over body,internal itching,movement stiffness,knee pain,hip joint pain,dischromic patches
0,Acne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hyperthyroidism,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Chronic cholestasis,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,Psoriasis,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4916,Peptic ulcer diseae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4917,Dengue,0,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4918,Fungal infection,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [86]:
dfx.drop(dfx.columns[-4:], axis=1, inplace=True)
dfx

,Disease,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,...,weakness of one body side,altered sensorium,nodal skin eruptions,red sore around nose,yellow crust ooze,ulcers on tongue,spotting urination,pain behind the eyes,red spots over body,internal itching
0,Acne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acne,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hyperthyroidism,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Chronic cholestasis,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,Psoriasis,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4916,Peptic ulcer diseae,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4917,Dengue,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4918,Fungal infection,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [87]:
columns_to_drop = ['foul_smell_ofurine', 'dischromic_patches', 'spotting_urination']
dfx = dfx.drop(columns=columns_to_drop)

In [88]:
dfx[dfx.columns[1:]].sum(axis=0).sort_values()

swelling_joints        0
muscle_weakness        0
hip_joint_pain         0
knee_pain              0
slurred_speech         0
                    ... 
nausea              1146
loss of appetite    1152
high fever          1362
vomiting            1914
fatigue             1932
Length: 222, dtype: int64

In [89]:
y=df['Disease'].unique()
y

array(['Acne', 'Hyperthyroidism', 'AIDS', 'Chronic cholestasis',
       'Hypertension', 'Hypoglycemia', 'Arthritis', 'Hepatitis B',
       'Migraine', 'Urinary tract infection', 'Diabetes', 'Hepatitis D',
       'Psoriasis', 'Alcoholic hepatitis', 'Dimorphic hemmorhoids(piles)',
       'Hepatitis E', 'Cervical spondylosis', 'Bronchial Asthma',
       'hepatitis A', 'Allergy', 'Hepatitis C', 'Pneumonia',
       'Hypothyroidism', 'Gastroenteritis', 'Varicose veins', 'Jaundice',
       'Drug Reaction', '(vertigo) Paroymsal  Positional Vertigo',
       'Heart attack', 'Tuberculosis', 'Typhoid', 'Common Cold',
       'Peptic ulcer diseae', 'Paralysis (brain hemorrhage)',
       'Fungal infection', 'Impetigo', 'GERD', 'Dengue', 'Malaria',
       'Chicken pox', 'Osteoarthristis'], dtype=object)

In [90]:
data = dfx.iloc[:,1:].values
labels = dfx['Disease'].values

In [91]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, train_size = 0.7,random_state=42)
x_train, x_val, y_train,y_val=train_test_split(data,labels,test_size=0.3,random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape, x_val.shape,y_val.shape)

(3444, 222) (1476, 222) (3444,) (1476,) (1476, 222) (1476,)


In [92]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_val=le.transform(y_val)

In [93]:
y=le.classes_
y

array(['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne',
       'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma',
       'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis',
       'Common Cold', 'Dengue', 'Diabetes',
       'Dimorphic hemmorhoids(piles)', 'Drug Reaction',
       'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Hypertension', 'Hyperthyroidism', 'Hypoglycemia',
       'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine',
       'Osteoarthristis', 'Paralysis (brain hemorrhage)',
       'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis',
       'Typhoid', 'Urinary tract infection', 'Varicose veins',
       'hepatitis A'], dtype=object)

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import pickle

# Assuming you've already loaded your dataset into DataFrames x_train, x_test, x_val, y_train, y_test, y_val

classifiers = {
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'LightGBM': LGBMClassifier(verbose=-1),
    'CatBoost': CatBoostClassifier(silent=True),
    'GradientBoost': GradientBoostingClassifier(),
    'ExtraTrees': ExtraTreesClassifier()
}

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

# K-fold Cross Validation model evaluation
for name, clf in classifiers.items():
    cv_scores = cross_val_score(clf, x_train, y_train, cv=kfold, scoring='f1_weighted')
    print(f'{name} cross-validation mean F1 score: %.3f' % cv_scores.mean())
    
    # Train and test each classifier
    clf.fit(x_train, y_train)
    
    test_predictions = clf.predict(x_test)
    test_f1 = f1_score(y_test, test_predictions, average='weighted')
    test_roc = roc_auc_score(y_test, clf.predict_proba(x_test), multi_class='ovr')
    print(f'{name} test F1 Score: {test_f1:.4f}, AUC-ROC Score: {test_roc:.4f}')
    
    val_predictions = clf.predict(x_val)
    val_f1 = f1_score(y_val, val_predictions, average='weighted')
    val_roc = roc_auc_score(y_val, clf.predict_proba(x_val), multi_class='ovr')
    print(f'{name} validation F1 Score: {val_f1:.4f}, AUC-ROC Score: {val_roc:.4f}')
    pickle.dump(clf, open(f"{name}_model.pkl", "wb"))

Random Forest cross-validation mean F1 score: 1.000
Random Forest test F1 Score: 1.0000, AUC-ROC Score: 1.0000
Random Forest validation F1 Score: 1.0000, AUC-ROC Score: 1.0000
XGBoost cross-validation mean F1 score: 0.999
XGBoost test F1 Score: 1.0000, AUC-ROC Score: 1.0000
XGBoost validation F1 Score: 1.0000, AUC-ROC Score: 1.0000


KeyboardInterrupt: 

In [201]:
desc = pd.read_csv("./Data/symptom_Description.csv")

In [202]:
desc.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [203]:
prec = pd.read_csv("./Data/symptom_precaution.csv")

In [204]:
prec.head()

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [205]:
def predd(m, X):
    # Get probabilities for each class
    proba = m.predict_proba(X)

    # Get the indices and probabilities of the top 5 classes
    top5_idx = np.argsort(proba[0])[-5:][::-1]
    top5_proba = np.sort(proba[0])[-5:][::-1]

    # Get the names of the top 5 diseases
    top5_diseases = y[top5_idx]

    for i in range(5):
        
        disease = top5_diseases[i]
        probability = top5_proba[i]
        # print(f"{disease}={probability}" )
        
        print("Disease Name: ", disease)
        print("Probability: ", probability)
        if(disease in desc["Disease"].unique()):
            disp = desc[desc['Disease'] == disease]
            disp = disp.values[0][1]
            print("Disease Description: ", disp)
        
        if(disease in prec["Disease"].unique()):
            c = np.where(prec['Disease'] == disease)[0][0]
            precuation_list = []
            for j in range(1, len(prec.iloc[c])):
                precuation_list.append(prec.iloc[c, j])
            print("Recommended Things to do at home: ")
            for precaution in precuation_list:
                print(precaution)
        
        print("\n")


In [206]:
prec

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths
5,GERD,avoid fatty spicy food,avoid lying down after eating,maintain healthy weight,exercise
6,Chronic cholestasis,cold baths,anti itch medicine,consult doctor,eat healthy
7,hepatitis A,Consult nearest hospital,wash hands through,avoid fatty spicy food,medication
8,Osteoarthristis,acetaminophen,consult nearest hospital,follow up,salt baths
9,(vertigo) Paroymsal Positional Vertigo,lie down,avoid sudden change in body,avoid abrupt head movment,relax


In [207]:
x=dfx.columns[1:]

In [208]:
x

Index(['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing',
       'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity',
       'ulcers_on_tongue',
       ...
       'weakness of one body side', 'altered sensorium',
       'nodal skin eruptions', 'red sore around nose', 'yellow crust ooze',
       'ulcers on tongue', 'spotting  urination', 'pain behind the eyes',
       'red spots over body', 'internal itching'],
      dtype='object', length=222)

In [209]:
y

array(['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne',
       'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma',
       'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis',
       'Common Cold', 'Dengue', 'Diabetes',
       'Dimorphic hemmorhoids(piles)', 'Drug Reaction',
       'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Hypertension', 'Hyperthyroidism', 'Hypoglycemia',
       'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine',
       'Osteoarthristis', 'Paralysis (brain hemorrhage)',
       'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis',
       'Typhoid', 'Urinary tract infection', 'Varicose veins',
       'hepatitis A'], dtype=object)

In [210]:
import pickle

In [211]:
t=pd.Series([0]*222, index=x)
m=ExtraTreesClassifier()
with open("./ExtraTrees_model.pkl", 'rb') as f:
    m =  pickle.load(f)
t.loc["chest_pain"]=1
t.loc["headache"]=1
t.loc["runny_nose"]=1
t.loc["high_fever"]=1
t.loc["throat_irritation"]=1
t.loc["congestion"]=1
t.loc["redness_of_eyes"]=1
t=t.to_numpy()
print(t.shape)
t=t.reshape(1,-1)
predd(m,t)

(222,)
Disease Name:  Paralysis (brain hemorrhage)
Probability:  0.24
Disease Description:  Intracerebral hemorrhage (ICH) is when blood suddenly bursts into brain tissue, causing damage to your brain. Symptoms usually appear suddenly during ICH. They include headache, weakness, confusion, and paralysis, particularly on one side of your body.
Recommended Things to do at home: 
massage
eat healthy
exercise
consult doctor


Disease Name:  Hypertension
Probability:  0.13
Disease Description:  Hypertension (HTN or HT), also known as high blood pressure (HBP), is a long-term medical condition in which the blood pressure in the arteries is persistently elevated. High blood pressure typically does not cause symptoms.


Disease Name:  Common Cold
Probability:  0.12
Disease Description:  The common cold is a viral infection of your nose and throat (upper respiratory tract). It's usually harmless, although it might not feel that way. Many types of viruses can cause a common cold.
Recommended Thi

In [106]:
import pickle
import numpy as np
import pandas as pd


with open("./ExtraTrees_model.pkl", 'rb') as f:
    model = pickle.load(f)

# Load the symptom descriptions and precautions
desc = pd.read_csv("./Data/symptom_Description.csv")
prec = pd.read_csv("./Data/symptom_precaution.csv")


# Create a binary vector for symptoms
t = pd.Series([0] * len(x), index=x)
t.loc["chest_pain"] = 1
t.loc["headache"] = 1
t.loc["runny_nose"] = 1
t.loc["congestion"] = 1

# Convert to numpy array and reshape
t = t.to_numpy()
t = t.reshape(1, -1)

# Predict the disease
prediction = model.predict_proba(t)
top5_idx = np.argsort(prediction[0])[-5:][::-1]
top5_proba = np.sort(prediction[0])[-5:][::-1]
top5_diseases = model.classes_[top5_idx]

# Display the results
for i in range(5):
    disease = top5_diseases[i]
    probability = top5_proba[i]
    print(f"Disease Name: {disease}")
    print(f"Probability: {probability:.2f}")
    
    # Display disease description
    if disease in desc["Disease"].unique():
        disp = desc[desc['Disease'] == disease].values[0][1]
        print(f"Disease Description: {disp}")
    
    # Display precautions
    if disease in prec["Disease"].unique():
        c = np.where(prec['Disease'] == disease)[0][0]
        precuation_list = []
        for j in range(1, len(prec.iloc[c])):
            precuation_list.append(prec.iloc[c, j])
        print("Recommended Things to do at home: ")
        for precaution in precuation_list:
            print(precaution)
    
    print("\n")

Disease Name: 32
Probability: 0.24


Disease Name: 23
Probability: 0.13


Disease Name: 10
Probability: 0.12


Disease Name: 15
Probability: 0.10


Disease Name: 38
Probability: 0.06




In [109]:
import tkinter as tk
from tkinter import ttk
import pickle
import numpy as np
import pandas as pd

# Load the trained model
with open("./ExtraTrees_model.pkl", 'rb') as f:
    model = pickle.load(f)

# Load the symptom descriptions and precautions
desc = pd.read_csv("./Data/symptom_Description.csv")
prec = pd.read_csv("./Data/symptom_precaution.csv")

# List of all symptoms (replace `x` with the actual list of symptoms from your dataset)
# Assuming `x` is the list of all 222 symptoms

t = pd.Series([0] * len(x), index=x)

# Function to predict disease
def predict_disease():
    # Get user input (symptoms separated by commas)
    user_input = symptom_entry.get().strip()
    if not user_input:
        result_text.delete(1.0, tk.END)
        result_text.insert(tk.END, "Please enter symptoms.")
        return
    
    # Split the input into individual symptoms
    selected_symptoms = [s.strip() for s in user_input.split(",")]
    
    # Create a binary vector for symptoms
    t = pd.Series([0] * len(x), index=x)
    for symptom in selected_symptoms:
        if symptom in x:
            t.loc[symptom] = 1
    
    # Convert to numpy array and reshape
    t = t.to_numpy()
    t = t.reshape(1, -1)
    
    # Predict the disease
    prediction = model.predict_proba(t)
    top5_idx = np.argsort(prediction[0])[-5:][::-1]
    top5_proba = np.sort(prediction[0])[-5:][::-1]
    top5_diseases = model.classes_[top5_idx]
    
    # Display the results
    result_text.delete(1.0, tk.END)
    for i in range(5):
        disease = top5_diseases[i]
        probability = top5_proba[i]
        result_text.insert(tk.END, f"Disease Name: {disease}\n")
        result_text.insert(tk.END, f"Probability: {probability:.2f}\n")
        
        # Display disease description
        if disease in desc["Disease"].unique():
            disp = desc[desc['Disease'] == disease].values[0][1]
            result_text.insert(tk.END, f"Disease Description: {disp}\n")
        
        # Display precautions
        if disease in prec["Disease"].unique():
            c = np.where(prec['Disease'] == disease)[0][0]
            precuation_list = []
            for j in range(1, len(prec.iloc[c])):
                precuation_list.append(prec.iloc[c, j])
            result_text.insert(tk.END, "Recommended Things to do at home: \n")
            for precaution in precuation_list:
                result_text.insert(tk.END, f"{precaution}\n")
        
        result_text.insert(tk.END, "\n")

# Create the main window
root = tk.Tk()
root.title("Disease Prediction")

# Create a frame for symptom input
input_frame = ttk.LabelFrame(root, text="Enter Symptoms (comma-separated)")
input_frame.grid(row=0, column=0, padx=10, pady=10)

# Create an entry widget for symptom input
symptom_entry = ttk.Entry(input_frame, width=80)
symptom_entry.grid(row=0, column=0, padx=10, pady=10)

# Create a button to predict the disease
predict_button = ttk.Button(root, text="Predict Disease", command=predict_disease)
predict_button.grid(row=1, column=0, pady=10)

# Create a text widget to display the results
result_text = tk.Text(root, height=20, width=80)
result_text.grid(row=2, column=0, padx=10, pady=10)

# Run the application
root.mainloop()

In [189]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import pickle
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier

# Load your trained ExtraTrees model
with open("./ExtraTrees_model.pkl", 'rb') as f:
    m = pickle.load(f)

# Dummy predict function (replace this with your own function if needed)
def predd(model, input_vector):
    return model.predict(input_vector)[0]

# GUI setup
root = tk.Tk()
root.title("Disease Prediction System")
root.geometry("550x300")

# Label
tk.Label(root, text="Enter symptoms (comma-separated):", font=('Arial', 13)).pack(pady=10)

# Input field
symptom_entry = tk.Entry(root, width=70)
symptom_entry.pack(pady=5)

# Output Label
output_label = tk.Label(root, text="", font=('Arial', 14, 'bold'), fg="blue")
output_label.pack(pady=20)

# Predict function
def predict():
    symptoms_input = symptom_entry.get()
    if not symptoms_input:
        messagebox.showwarning("Input Error", "Please enter some symptoms.")
        return

    # Process symptoms
    input_symptoms = [s.strip().lower().replace(" ", "_") for s in symptoms_input.split(',')]

    # Create a Series of zeros
    t = pd.Series([0] * len(x), index=x)

    for symptom in input_symptoms:
        if symptom in t.index:
            t.loc[symptom] = 1
        else:
            messagebox.showwarning("Invalid Symptom", f"'{symptom}' not found in symptom list.")
            return

    # Convert to numpy and reshape
    t = t.to_numpy().reshape(1, -1)

    try:
        prediction = predd(m, t)
        output_label.config(text=f"Predicted Disease: {prediction}")
    except Exception as e:
        messagebox.showerror("Prediction Error", str(e))

# Predict Button
tk.Button(root, text="Predict", font=('Arial', 12), command=predict, bg="lightgreen").pack(pady=10)

root.mainloop()


In [221]:
import pandas as pd
import pickle

# Load the trained model
m = None
with open("./ExtraTrees_model.pkl", 'rb') as f:
    m = pickle.load(f)

t = pd.Series([0] * len(x), index=x)

# Prompt the user to enter symptoms
print("Enter your symptoms (comma-separated):")
user_symptoms = input().strip().split(',')

# Map user symptoms to the feature vector
for symptom in user_symptoms:
    symptom = symptom.strip()
    if symptom in t.index:
        t.loc[symptom] = 1

# Convert the Series to a numpy array and reshape for prediction
t = t.to_numpy().reshape(1, -1)

t=t.reshape(1,-1)

predd(m,t)


Enter your symptoms (comma-separated):
Disease Name:  Paralysis (brain hemorrhage)
Probability:  0.27
Disease Description:  Intracerebral hemorrhage (ICH) is when blood suddenly bursts into brain tissue, causing damage to your brain. Symptoms usually appear suddenly during ICH. They include headache, weakness, confusion, and paralysis, particularly on one side of your body.
Recommended Things to do at home: 
massage
eat healthy
exercise
consult doctor


Disease Name:  Hypertension
Probability:  0.16
Disease Description:  Hypertension (HTN or HT), also known as high blood pressure (HBP), is a long-term medical condition in which the blood pressure in the arteries is persistently elevated. High blood pressure typically does not cause symptoms.


Disease Name:  Fungal infection
Probability:  0.12
Disease Description:  In humans, fungal infections occur when an invading fungus takes over an area of the body and is too much for the immune system to handle. Fungi can live in the air, soil, w

In [223]:
import pandas as pd
import pickle

# Load the trained model
m = None
with open("./ExtraTrees_model.pkl", 'rb') as f:
    m = pickle.load(f)

# Initialize a Series with zeros, using the predefined 'x' (feature names)
t = pd.Series([0] * len(x), index=x)

# Prompt the user to enter symptoms
print("Enter your symptoms (comma-separated):")
user_symptoms = input().strip().split(',')

# Map user symptoms to the feature vector
for symptom in user_symptoms:
    symptom = symptom.strip()
    if symptom in t.index:
        t.loc[symptom] = 1

# Convert the Series to a numpy array and reshape for prediction
t = t.to_numpy().reshape(1, -1)

# Predict the diseases
predicted_diseases = m.predict(t)
print("Predicted Diseases:", predicted_diseases)

Enter your symptoms (comma-separated):


Predicted Diseases: [32]
